<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/tweet_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xlrd
import re
import pickle

In [2]:
# Uncomment if you're running it for the first time
# !pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [3]:
# Uncomment if you're running it for the first time
# !pip install tweet-preprocessor
import preprocessor as tweet_proc

In [4]:
# Uncomment if you're running it for the first time
# !pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

#### *Raw Datasets are hosted [here](https://drive.google.com/drive/folders/1TuHRQQ41lK9oXJhlhsRiMjMnczuKn_kF?usp=sharing).*

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [6]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [7]:
# For 2020 Datasets
# For English
file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/english.xlsx"

# For Hindi
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/hindi.xlsx"

# For German
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/german.xlsx"

In [8]:
# Initializing Lists
datapoints_count = 0
see_index = True

tweets = []
raw_tweet_texts = []

hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []

task_1_labels = []
task_2_labels = []
tweet_ids = []
hasoc_ID = []

In [9]:
workbook = xlrd.open_workbook(file_name)
sheet = workbook.sheet_by_index(0)

count = 0
for row in range(sheet.nrows):
    line = sheet.row_values(row)

    if see_index == True:
        see_index = False
        continue

    datapoints_count += 1
    tweet_ids.append(line[0])
    task_1_labels.append(line[2])
    task_2_labels.append(line[3])
    hasoc_ID.append(line[4])
    tweets.append(line[1])
    raw_tweet_texts.append(tweet_proc.clean(line[1]))

    parse_obj = tweet_proc.parse(line[1])

    hashtags.append(make_list(parse_obj.hashtags))
    smileys.append(make_list(parse_obj.smileys))
    emojis.append(make_list(parse_obj.emojis))
    urls.append(make_list(parse_obj.urls))
    mentions.append(make_list(parse_obj.mentions))
    numbers.append(make_list(parse_obj.numbers))
    reserveds.append(make_list(parse_obj.reserved))

print("Number of Datapoints: " + str(datapoints_count))
    

Number of Datapoints: 3708


In [10]:
# Viewing Created Dataset

print("Tweets:")
print(tweets[0: 5])

print("Raw Texts:")
print(raw_tweet_texts[0: 5])

print("Hashtags:")
print(hashtags[0: 5])

print("Smileys:")
print(smileys[0: 5])

print("Emojis:")
print(emojis[0: 5])

print("Urls:")
print(urls[0: 5])

print("Mentions:")
print(mentions[0: 5])

print("Numbers:")
print(numbers[0: 5])

print("Reserved Words:")
print(reserveds[0: 5])

print("Task Labels:")
print(task_1_labels[0: 5])
print(task_2_labels[0: 5])

Tweets:
['hate wen females hit ah nigga with tht bro 😂😂, I’m tryna make u my la sweety , fuck ah bro', "RT @airjunebug: When you're from the Bay but you're really a NY nigga at heart. W/ @supportcaleon https://t.co/mZ8BAYlnlf", 'RT @DonaldJTrumpJr: Dear Democrats: The American people aren’t stupid, they know what spying is and no amount of gaslighting will change th…', 'RT @SheLoveTimothy: He ain’t on drugs he just bored. I be doing the same shit when I’m bored 😂 https://t.co/tkdjSbddET', 'RT @TavianJordan: Summer ‘19 I’m coming for you ! No boring shit ! Beach days, road trips, kickbacks and HOT DAYS ! I’m ready I’m ready I’m…']
Raw Texts:
['hate wen females hit ah nigga with tht bro , Im tryna make u my la sweety , fuck ah bro', ": When you're from the Bay but you're really a NY nigga at heart. W/", ': Dear Democrats: The American people arent stupid, they know what spying is and no amount of gaslighting will change th', ': He aint on drugs he just bored. I be doing the same shit whe

In [11]:
# Generating Emoji Texts
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text.replace("_", " "))
  emoji_texts.append(texts)

print("Emoji Descriptions:")
print(emoji_texts[0: 5])

Emoji Descriptions:
[['face with tears of joy', 'face with tears of joy'], [], [], ['face with tears of joy'], []]


In [12]:
# Segmenting Hashtags
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

print("Segmented Hashtags:")
print(segmented_hashtags[0: 5])

Segmented Hashtags:
[[], [], [], [], []]


In [13]:
name = 'ge.pickle'
dickie = {}
dickie['tweet_id'] = tweet_ids
dickie['task_1'] = task_1_labels
dickie['task_2'] = task_2_labels
dickie['hasoc_id'] = hasoc_ID
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags

with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [14]:
with open(name, 'rb') as f:
  try_dict = pickle.load(f)

sizes = []
for key in try_dict.keys():
  sizes.append(len(try_dict[key]))

# Verifying if all sizes are equal
print(sizes)

[3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708, 3708]


In [15]:
# ^_^ Thank You